# BayesNet

A `BayesNet` in GTSAM represents a directed graphical model, specifically the result of running sequential variable elimination (like Cholesky or QR factorization) on a `FactorGraph`.

It is essentially a collection of `Conditional` objects, ordered according to the elimination order. Each conditional represents $P(	ext{variable} | 	ext{parents})$, where the parents are variables that appear later in the elimination ordering.

Like `FactorGraph`, `BayesNet` is templated on the type of conditional it stores (e.g., `GaussianBayesNet`, `DiscreteBayesNet`).

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/inference/doc/BayesNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

In [ ]:
import gtsam
import numpy as np

# We need concrete graph types and elimination to get a BayesNet
from gtsam import GaussianFactorGraph, Ordering, GaussianBayesNet
from gtsam import symbol_shorthand

X = symbol_shorthand.X
L = symbol_shorthand.L

## Creating a BayesNet (via Elimination)

BayesNets are typically obtained by eliminating a `FactorGraph`.

In [ ]:
# Create a simple Gaussian Factor Graph P(x0) P(x1|x0) P(x2|x1)
graph = GaussianFactorGraph()
model = gtsam.noiseModel.Isotropic.Sigma(1, 1.0)
graph.add(X(0), -np.eye(1), np.zeros(1), model)
graph.add(X(0), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(X(1), -np.eye(1), X(2), np.eye(1), np.zeros(1), model)
print("Original Factor Graph:")
graph.print()

# Eliminate sequentially using a specific ordering
ordering = Ordering([X(0), X(1), X(2)])
bayes_net = graph.eliminateSequential(ordering)

print("\nResulting BayesNet:")
bayes_net.print()

Original Factor Graph: size 3
Factor 0: JacobianFactor(keys = [8070450532247928832], Z = [ -1 ], b = [ 0 ], model = diagonal sigmas [1])
Factor 1: JacobianFactor(keys = [8070450532247928832; 8070450532247928833], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])
Factor 2: JacobianFactor(keys = [8070450532247928833; 8070450532247928834], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])


Resulting BayesNet: size 3
Conditional 0: GaussianConditional( P(x0 | x1) = dx0 - R*dx1 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.707107 ])
Conditional 1: GaussianConditional( P(x1 | x2) = dx1 - R*dx2 - d), R = [ 0.666667 ], d = [ 0 ], sigmas = [ 0.816497 ])
Conditional 2: GaussianConditional( P(x2) = dx2 - d), d = [ 0 ], sigmas = [ 0.866025 ])



## Properties and Access

A `BayesNet` provides access to its constituent conditionals and basic properties.

In [ ]:
print(f"BayesNet size: {bayes_net.size()}")
print(f"Is BayesNet empty? {bayes_net.empty()}")

# Access conditional by index
conditional1 = bayes_net.at(1)
print("Conditional at index 1: ")
conditional1.print()

# Get all keys involved
bn_keys = bayes_net.keys()
print(f"Keys in BayesNet: {bn_keys}")

# Iterate through conditionals
# for i, conditional in enumerate(bayes_net):
#     if conditional:
#         print(f"Conditional {i} Frontals: {conditional.frontals()}, Parents: {conditional.parents()}")

BayesNet size: 3
Is BayesNet empty? False
Conditional at index 1: 
GaussianConditional( P(x1 | x2) = dx1 - R*dx2 - d), R = [ 0.666667 ], d = [ 0 ], sigmas = [ 0.816497 ])

Keys in BayesNet: {8070450532247928832, 8070450532247928833, 8070450532247928834}


## Evaluation and Solution

The `logProbability(Values)` method computes the log probability of a variable assignment given the conditional distributions in the Bayes net. For Gaussian Bayes nets, the `optimize()` method can be used to find the maximum likelihood estimate (MLE) solution via back-substitution.

In [ ]:
# For GaussianBayesNet, we use VectorValues
mle_solution = bayes_net.optimize()

# Calculate log probability (requires providing values for all variables)
log_prob = bayes_net.logProbability(mle_solution)
print(f"Log Probability at {mle_solution.at(X(0))[0]:.0f},{mle_solution.at(X(1))[0]:.0f},{mle_solution.at(X(2))[0]:.0f}]: {log_prob}")

print("Optimized Solution (MLE):")
mle_solution.print()

Log Probability at [0,0,0]: -2.756815765004782
Optimized Solution (MLE):
Values with 3 values:
Value x0: [0.]
Value x1: [0.]
Value x2: [0.]



## Visualization

Bayes nets can also be visualized using Graphviz.

In [ ]:
dot_string = bayes_net.dot()
print(dot_string)

# To render:
# dot -Tpng bayesnet.dot -o bayesnet.png
# import graphviz
# graphviz.Source(dot_string)

digraph {
  size="5,5";

  var8070450532247928832[label="x0"];
  var8070450532247928833[label="x1"];
  var8070450532247928834[label="x2"];

  var8070450532247928834->var8070450532247928833
  var8070450532247928833->var8070450532247928832
}
